# Momentum ETF Performance

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.graph_objects as go
import statsmodels.api as sm
pd.options.display.float_format = '{:.4f}'.format

In [2]:
# Function to pull returns
def returns(tickers):
    ret = yf.download(tickers, start='2000-01-01', end='2022-12-31', progress=False)
    ret.index = ret.index.to_period('D')
    ret = ret["Adj Close"].resample("M").last()
    ret = ret.pct_change()
    ret.columns = tickers
    return ret

In [3]:
# Pull data
ticker_list = ['MTUM','PDP','XMMO','ONEO','VFMO','JMOM','SPMO','QMOM','FDMO','MMTM']

# Pull the data from Yahoo
df = returns(ticker_list)
df

d:\bbcx-investments-quarto\quarto\venv\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


,MTUM,PDP,XMMO,ONEO,VFMO,JMOM,SPMO,QMOM,FDMO,MMTM
Date,,,,,,,,,,
2005-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0368
2005-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0678
2005-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0432
2005-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0343
...,...,...,...,...,...,...,...,...,...,...
2022-08,-0.0266,-0.0338,-0.0302,-0.0199,-0.0241,-0.0201,0.0296,-0.0295,-0.0039,-0.0320
2022-09,-0.0743,-0.0867,-0.0803,-0.0595,-0.0926,-0.0863,-0.0783,-0.0698,-0.0764,-0.0960
2022-10,0.0849,0.0873,0.0962,0.1255,0.1121,0.1061,0.1572,0.1358,0.1287,0.1215


In [4]:
# Add factors
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
Mom = pdr('F-F_Momentum_Factor','famafrench', start=1900)[0]/100
Mom.columns = ['Mom']
ff = ff3.join(Mom)
df = df.join(ff)

## Run market model and Fama-French-Carhart performance models

In [5]:
stats = pd.DataFrame(dtype=float, columns=pd.MultiIndex.from_product([['CAPM','FF4'],['alpha','beta','t_alpha','t_beta','nobs']]), index=ticker_list)
stats

CAPM                            FF4                         
     alpha beta t_alpha t_beta nobs alpha beta t_alpha t_beta nobs
MTUM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
PDP    NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
XMMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
ONEO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
VFMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
JMOM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
SPMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
QMOM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
FDMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
MMTM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN

In [6]:
# Run the performance evaluations
for tick in ticker_list:
    # Market-model
    mm = sm.OLS(df[tick]-df['RF'], sm.add_constant(df['Mkt-RF']),missing='drop').fit()
    stats.loc[tick, ('CAPM','alpha')] = mm.params[0]
    stats.loc[tick, ('CAPM','beta')]  = mm.params[1]
    stats.loc[tick, ('CAPM','t_alpha')] = mm.tvalues[0]
    stats.loc[tick, ('CAPM','t_beta')]  = mm.tvalues[1]
    stats.loc[tick, ('CAPM','nobs')]  = mm.nobs

    # Fama-French 4-factor model
    ff4 = sm.OLS(df[tick]-df['RF'], sm.add_constant(df[['Mkt-RF','SMB','HML','Mom']]),missing='drop').fit()
    stats.loc[tick, ('FF4','alpha')] = ff4.params[0]
    stats.loc[tick, ('FF4','beta')]  = ff4.params[1]
    stats.loc[tick, ('FF4','coeff_smb')]  = ff4.params[2]
    stats.loc[tick, ('FF4','coeff_hml')]  = ff4.params[3]
    stats.loc[tick, ('FF4','coeff_mom')]  = ff4.params[4]
    stats.loc[tick, ('FF4','t_alpha')] = ff4.tvalues[0]
    stats.loc[tick, ('FF4','t_beta')]  = ff4.tvalues[1]  
    stats.loc[tick, ('FF4','nobs')]    = ff4.nobs 
stats

CAPM                                     FF4                         \
       alpha   beta t_alpha  t_beta     nobs   alpha   beta t_alpha  t_beta   
MTUM -0.0005 0.9378 -0.3235 30.8076  75.0000 -0.0020 1.0201 -1.7842 41.9383   
PDP   0.0007 0.9826  0.4031 31.7898  61.0000 -0.0006 1.0486 -0.4598 42.8383   
XMMO  0.0008 0.9237  0.7202 39.8148 122.0000 -0.0002 0.9841 -0.2525 43.2379   
ONEO  0.0020 0.9017  1.1085 23.1686 116.0000 -0.0002 1.0211 -0.1691 34.0169   
VFMO -0.0019 1.0333 -1.2152 33.3102  84.0000 -0.0019 1.0172 -1.5246 36.1108   
JMOM -0.0009 1.0178 -0.6213 33.1461 189.0000 -0.0020 1.0923 -1.6113 38.2897   
SPMO -0.0007 1.1103 -0.1491 12.5357  84.0000 -0.0021 1.2097 -0.6248 15.4399   
QMOM  0.0026 0.8691  1.2604 20.3881  86.0000  0.0008 1.0082  0.5356 27.8954   
FDMO  0.0010 1.0130  0.3338 19.0370  58.0000 -0.0001 1.0391 -0.0617 28.1661   
MMTM  0.0010 1.0406  0.6477 30.8793 213.0000  0.0004 1.0627  0.2984 32.3917   

                                             
         nobs coeff_smb coeff_hml coeff_mom  
MTUM  75.0000   -0.0014   -0.0790    0.2327  
PDP   61.0000    0.0109   -0.1063    0.1934  
XMMO 122.0000   -0.1419   -0.0050    0.1188  
ONEO 116.0000   -0.0616   -0.1210    0.3311  
VFMO  84.0000    0.1087    0.2145    0.0297  
JMOM 189.0000    0.1235   -0.1680    0.1940  
SPMO  84.0000    0.7800    0.0860    0.6729  
QMOM  86.0000   -0.1052   -0.0431    0.3428  
FDMO  58.0000    0.6077    0.1571    0.3730  
MMTM 213.0000    0.3186   -0.1284    0.1762

In [7]:
# Sort to find the highest historical alpha
stats.sort_values(by=[('CAPM','t_alpha')], ascending=False)

CAPM                                     FF4                         \
       alpha   beta t_alpha  t_beta     nobs   alpha   beta t_alpha  t_beta   
QMOM  0.0026 0.8691  1.2604 20.3881  86.0000  0.0008 1.0082  0.5356 27.8954   
ONEO  0.0020 0.9017  1.1085 23.1686 116.0000 -0.0002 1.0211 -0.1691 34.0169   
XMMO  0.0008 0.9237  0.7202 39.8148 122.0000 -0.0002 0.9841 -0.2525 43.2379   
MMTM  0.0010 1.0406  0.6477 30.8793 213.0000  0.0004 1.0627  0.2984 32.3917   
PDP   0.0007 0.9826  0.4031 31.7898  61.0000 -0.0006 1.0486 -0.4598 42.8383   
FDMO  0.0010 1.0130  0.3338 19.0370  58.0000 -0.0001 1.0391 -0.0617 28.1661   
SPMO -0.0007 1.1103 -0.1491 12.5357  84.0000 -0.0021 1.2097 -0.6248 15.4399   
MTUM -0.0005 0.9378 -0.3235 30.8076  75.0000 -0.0020 1.0201 -1.7842 41.9383   
JMOM -0.0009 1.0178 -0.6213 33.1461 189.0000 -0.0020 1.0923 -1.6113 38.2897   
VFMO -0.0019 1.0333 -1.2152 33.3102  84.0000 -0.0019 1.0172 -1.5246 36.1108   

                                             
         nobs coeff_smb coeff_hml coeff_mom  
QMOM  86.0000   -0.1052   -0.0431    0.3428  
ONEO 116.0000   -0.0616   -0.1210    0.3311  
XMMO 122.0000   -0.1419   -0.0050    0.1188  
MMTM 213.0000    0.3186   -0.1284    0.1762  
PDP   61.0000    0.0109   -0.1063    0.1934  
FDMO  58.0000    0.6077    0.1571    0.3730  
SPMO  84.0000    0.7800    0.0860    0.6729  
MTUM  75.0000   -0.0014   -0.0790    0.2327  
JMOM 189.0000    0.1235   -0.1680    0.1940  
VFMO  84.0000    0.1087    0.2145    0.0297

In [8]:
alpha = stats.loc['SPMO',('CAPM','alpha')]*12
print(f'{alpha: .2%}')

-0.79%


In [9]:
ff4 = sm.OLS(df['SPMO']-df['RF'], sm.add_constant(df[['Mkt-RF','SMB','HML','Mom']]),missing='drop').fit()
print(ff4.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                     80.96
Date:                Mon, 10 Apr 2023   Prob (F-statistic):           3.71e-27
Time:                        12:49:16   Log-Likelihood:                 176.67
No. Observations:                  84   AIC:                            -343.3
Df Residuals:                      79   BIC:                            -331.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0021      0.003     -0.625      0.5